# **수상작 리뷰**

## 데이터셋 소개

주제 : AI vs Human 텍스트 판별 해커톤

* 처음 주어진 데이터는 NLI/NLU 형태.
* 문장 4개 중 정답에 해당하는 문장을 선택하는 구조.
* Binary Classification으로 재정의해 정답 문장은 1, 오답 문장은 0 형태로 데이터셋을 확장함.
* 결과적으로 Positive/Negative 샘플 균형이 생기고, 모델이 문장 하나 기준으로 예측하도록 단순화됨.

## 전처리 과정

* Text 기반 Tokenizer(AutoTokenizer, FunnelTokenizerFast) 사용

* max length padding, truncation, attention mask, token_type_ids 모두 포함해 모델 입력 형태 유지

* StratifiedKFold(5-fold)로 데이터 분리 → class imbalance 방지

* Label 방식 변경 후 train/test reshape 작업 수행
→ test셋도 melt/pivot 변환을 통해 binary inference → final multi-label 재구성

## 모델링

**단일 모델**

* 세 개의 한국어 언어모델 실험: RoBERTa / BERT-base / Funnel Transformer
* 동일한 optimizer, lr, seed 유지 → fair comparison
* 모든 모델 Pytorch Lightning 기반 pipeline 재사용
* 최종 layer에 SwiGLU + Linear 적용 → 기존 linear보다 일반화와 token representation 활용 강화됨

**Ensemble**

* Soft voting 방식 채택
* 조합별 Public/Private score 비교 후
* 최종적으로 A+B+C Soft Ensemble 선택

결과적으로 single model 대비 ensemble 모델이 성능적으로 우수했으며, 특히 B + C 조합이 가장 높은 private score를 기록했다는 점은 의미 있음.

## 차별점 및 배울점

RoBERTa, BERT, Funnel 구조 비교를 비교하고 일반적인 classifier 대신 activation 방식 변경하였다. 단순 평균이 아니라 확률 기반 합산으로 Soft Ensemble을 사용하였다.

Binary로 단순화된 문제 정의가 모델 정확도와 안정성에 영향을 미치고, Transformer 구조가 다르더라도 Soft Ensemble은 개별 모델의 약점을 상쇄할 수 있다는 점을 배웠다. 또한 Activation 변경(SwiGLU)이 classification head에서 성능 차이를 줄 수 있음을 알 수 있었다.